In [1]:
import pandas as pd
import scipy.optimize as opt

In [48]:
# read in demand data
demand = pd.read_csv("/content/citrus_demand.csv")

# read in supply predictions data
actual_supply = pd.read_csv("/content/citrus_var_predict.csv")

# read in prices data
wholesale_prices = pd.read_csv("/content/wholesale_prices_short.csv")
retail_prices = pd.read_csv("/content/est_retail_prices.csv")

In [49]:
demand = demand[demand["year"] >= 1998]
demand = demand[demand["year"] <= 2015]
demand.reset_index(inplace=True, drop=True)

demand.head()

,year,Chicago,Detroit,Minneapolis.StPaul,Boston,New.York,Philadelphia,Atlanta,Baltimore,Dallas.FortWorth,Houston,Miami,DC,Los.Angeles,San.Francisco,San.Diego,Seattle,Average
0,1998,1.870383e+08,8.603941e+07,6.388088e+07,7.978738e+07,4.370930e+08,8.868379e+07,5.992135e+07,4.446037e+07,9.843968e+07,7.708341e+07,8.109145e+07,1.089691e+08,2.297290e+08,1.290025e+08,5.881579e+07,6.936250e+07,1.117293e+08
1,1999,1.825194e+08,8.914297e+07,6.514830e+07,9.450791e+07,4.323608e+08,1.055312e+08,6.781170e+07,5.212063e+07,1.193869e+08,8.806082e+07,9.633104e+07,1.129938e+08,2.517448e+08,1.327160e+08,6.042776e+07,7.911058e+07,1.194067e+08
2,2000,1.671226e+08,9.295417e+07,6.957600e+07,9.300697e+07,4.918061e+08,1.252924e+08,7.166764e+07,5.099217e+07,1.368396e+08,9.114355e+07,1.138096e+08,1.190512e+08,2.679443e+08,1.326530e+08,5.406173e+07,9.445342e+07,1.277867e+08
3,2001,1.796239e+08,8.332047e+07,6.673847e+07,9.114897e+07,4.670425e+08,1.194820e+08,6.486827e+07,4.562551e+07,1.306799e+08,8.265417e+07,1.097253e+08,1.024777e+08,2.389385e+08,1.157599e+08,4.523408e+07,8.862140e+07,1.195260e+08
4,2002,1.741256e+08,7.768032e+07,6.735025e+07,9.495969e+07,4.347284e+08,9.585756e+07,7.335671e+07,4.983372e+07,1.160534e+08,8.097091e+07,1.056303e+08,1.059825e+08,2.447750e+08,1.046570e+08,4.883273e+07,8.052063e+07,1.150185e+08


In [50]:
actual_supply = actual_supply[actual_supply["year"] >= 1999]
actual_supply.reset_index(inplace=True, drop=True)

actual_supply.head()

,year,west_lbs,p_sw_lbs,mw_se_lbs,a_ne_lbs,west_half_lbs,east_half_lbs
0,1999,5.607182e+08,6.130679e+07,1.466984e+09,0,6.220250e+08,1.466984e+09
1,2000,2.731724e+08,2.496791e+07,6.801667e+08,0,2.981403e+08,6.801667e+08
2,2001,4.187099e+08,4.204311e+07,1.055122e+09,0,4.607530e+08,1.055122e+09
3,2002,4.734132e+08,4.852686e+07,1.195760e+09,0,5.219400e+08,1.195760e+09
4,2003,5.029862e+08,5.108567e+07,1.250946e+09,0,5.540718e+08,1.250946e+09


In [51]:
wholesale_prices = wholesale_prices[wholesale_prices["year"] >= 1999]
wholesale_prices = wholesale_prices[wholesale_prices["year"] <= 2015]
wholesale_prices.reset_index(inplace=True, drop=True)

wholesale_prices.head()

,year,Apples,Bananas,Grapes,PeachesNectarines,Strawberries,OtherBerries,Citrus,Melons,OtherFruit,...,Carrots,Broccoli,Spinach,Lettuce,StringBeans,Cabbage,Onions,Corn,OtherVeg,Potatoes
0,1999,21.3,35.0,33.00,28.6,74.4,102,17.250,11.101633,67.161655,...,16.8,24.1,32.1,15.098916,46.5,11.3,9.7,16.9,27.428610,6.9
1,2000,17.8,36.0,32.35,27.6,64.9,115,7.000,11.035726,79.001992,...,13.1,31.2,31.8,19.228349,42.6,12.3,11.2,18.5,28.129329,5.3
2,2001,22.9,38.0,34.50,30.7,75.8,109,9.625,11.863701,72.454395,...,17.1,26.5,32.4,19.287165,45.0,13.4,10.7,19.5,27.847922,10.8
3,2002,25.8,43.0,34.30,30.6,71.3,117,11.250,12.184179,87.104935,...,19.1,31.4,34.2,23.559195,47.6,12.9,12.1,19.2,27.546230,9.6
4,2003,26.1,41.0,34.45,29.1,74.9,120,8.125,12.388878,79.634487,...,19.0,32.7,36.9,21.827844,49.2,13.3,13.7,19.2,28.384431,7.3


In [52]:
retail_prices = retail_prices[retail_prices["year"] >= 1999]
retail_prices = retail_prices[retail_prices["year"] <= 2015]
retail_prices.reset_index(inplace=True, drop=True)

retail_prices.head()

,year,Apples,Bananas,Grapes,PeachesNectarines,Strawberries,OtherBerries,Citrus,Melons,OtherFruit,...,Carrots,Broccoli,Spinach,Lettuce,StringBeans,Cabbage,Onions,Corn,OtherVeg,Potatoes
0,1999,86.0,52.5,49.500,42.90,208,153.0,25.8750,16.652449,100.742482,...,25.20,36.15,48.15,22.648374,69.75,16.95,14.55,25.35,41.142914,10.35
1,2000,88.0,54.0,48.525,41.40,188,172.5,10.5000,16.553589,118.502987,...,19.65,46.80,47.70,28.842524,63.90,18.45,16.80,27.75,42.193994,7.95
2,2001,83.0,57.0,51.750,46.05,229,163.5,14.4375,17.795551,108.681593,...,25.65,39.75,48.60,28.930747,67.50,20.10,16.05,29.25,41.771883,16.20
3,2002,91.0,64.5,51.450,45.90,228,175.5,16.8750,18.276268,130.657402,...,28.65,47.10,51.30,35.338792,71.40,19.35,18.15,28.80,41.319345,14.40
4,2003,94.0,61.5,51.675,43.65,237,180.0,12.1875,18.583316,119.451730,...,28.50,49.05,55.35,32.741766,73.80,19.95,20.55,28.80,42.576646,10.95


In [53]:
# parameters
N = 16 # number of retailers
K = demand.shape[0]-1 # number of iterations (0th one is the initial condition)

# dataframes to keep track of orders
# columns represent retailers, rows represent iterations
orders_west = pd.DataFrame(columns=range(0,N), index=range(0,K+1))
orders_east = pd.DataFrame(columns=range(0,N), index=range(0,K+1))

# initial orders
for i in range(0,N):
  orders_west.iloc[0,i] = demand["Average"].iloc[0] / 2
  orders_east.iloc[0,i] = demand["Average"].iloc[0] / 2

In [54]:
def ret_cost_sim(q, k, curr_i, orders_west, orders_east, demand, supply, wholesale_prices, retail_prices):
  Q_i = demand["Average"].iloc[k+1]

  total_request_west = q[0]
  total_request_west += orders_west.iloc[k,0:curr_i].sum() + orders_west.iloc[k,curr_i+1:].sum()

  total_request_east = q[1]
  total_request_east += orders_east.iloc[k,0:curr_i].sum() + orders_east.iloc[k,curr_i+1:].sum()

  if (total_request_west <= supply["west_half_lbs"][k]):
    curr_R_west = q[0]
  else:
    curr_R_west = supply["west_half_lbs"][k] * (q[0] / total_request_west)

  if (total_request_east <= supply["east_half_lbs"][k]):
    curr_R_east = q[1]
  else:
    curr_R_east = supply["east_half_lbs"][k] * (q[1] / total_request_east)

  R = curr_R_west + curr_R_east

  holding = max(R - Q_i, 0)
  stockout = max(Q_i - R, 0)

  holding = wholesale_prices["Citrus"][k] * holding
  stockout = retail_prices["Citrus"][k] * stockout

  cost = holding + stockout
  return cost

In [55]:
for k in range(0,K):
  print(k)
  curr_i = 0 # retailers are identical

  rranges = (slice(10**5, 10**6, 10**5), slice(10**5, 10**6, 10**5))
  params_curr = (k, curr_i, orders_west, orders_east, demand, actual_supply, wholesale_prices, retail_prices)
  opt_sol = opt.brute(ret_cost_sim, rranges, args=params_curr, full_output=True)

  for i in range(0,N):
    if opt_sol[0][0] > 0:
      orders_west.iloc[k+1,i] = opt_sol[0][0]
    else:
      orders_west.iloc[k+1,i] = 0

    if opt_sol[0][1] > 0:
      orders_east.iloc[k+1,i] = opt_sol[0][1]
    else:
      orders_east.iloc[k+1,i] = 0

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [35]:
actual_supply["west_half_lbs"].iloc[0] / 16 / 10**7

3.887656391122369

In [34]:
demand["Average"]

0     1.117293e+08
1     1.194067e+08
2     1.277867e+08
3     1.195260e+08
4     1.150185e+08
5     1.018912e+08
6     1.057780e+08
7     1.261478e+08
8     1.227120e+08
9     1.167984e+08
10    1.149018e+08
11    1.194756e+08
12    1.076447e+08
13    1.225133e+08
14    1.318389e+08
15    1.152118e+08
16    1.243792e+08
17    1.238535e+08
Name: Average, dtype: float64

In [56]:
orders_total = orders_west + orders_east
(orders_total.iloc[:,0] - demand["Average"]) / 10**7

0          0.0
1     0.556152
2     9.213962
3     1.851457
4      0.62323
5     1.170855
6     0.292374
7     1.453023
8     1.025024
9     1.560449
10    0.870117
11    1.477704
12    0.841447
13    1.401718
14    0.956628
15    1.355883
16    0.585066
17    1.424014
dtype: object

In [ ]:
demand

,year,Chicago,Detroit,Minneapolis.StPaul,Boston,New.York,Philadelphia,Atlanta,Baltimore,Dallas.FortWorth,Houston,Miami,DC,Los.Angeles,San.Francisco,San.Diego,Seattle,Average
0,1998,1.870383e+08,8.603941e+07,6.388088e+07,7.978738e+07,4.370930e+08,8.868379e+07,5.992135e+07,4.446037e+07,9.843968e+07,7.708341e+07,8.109145e+07,1.089691e+08,2.297290e+08,1.290025e+08,5.881579e+07,6.936250e+07,1.117293e+08
1,1999,1.825194e+08,8.914297e+07,6.514830e+07,9.450791e+07,4.323608e+08,1.055312e+08,6.781170e+07,5.212063e+07,1.193869e+08,8.806082e+07,9.633104e+07,1.129938e+08,2.517448e+08,1.327160e+08,6.042776e+07,7.911058e+07,1.194067e+08
2,2000,1.671226e+08,9.295417e+07,6.957600e+07,9.300697e+07,4.918061e+08,1.252924e+08,7.166764e+07,5.099217e+07,1.368396e+08,9.114355e+07,1.138096e+08,1.190512e+08,2.679443e+08,1.326530e+08,5.406173e+07,9.445342e+07,1.277867e+08
3,2001,1.796239e+08,8.332047e+07,6.673847e+07,9.114897e+07,4.670425e+08,1.194820e+08,6.486827e+07,4.562551e+07,1.306799e+08,8.265417e+07,1.097253e+08,1.024777e+08,2.389385e+08,1.157599e+08,4.523408e+07,8.862140e+07,1.195260e+08
4,2002,1.741256e+08,7.768032e+07,6.735025e+07,9.495969e+07,4.347284e+08,9.585756e+07,7.335671e+07,4.983372e+07,1.160534e+08,8.097091e+07,1.056303e+08,1.059825e+08,2.447750e+08,1.046570e+08,4.883273e+07,8.052063e+07,1.150185e+08
5,2003,1.511506e+08,6.857064e+07,5.667943e+07,7.959782e+07,3.703894e+08,7.666855e+07,6.963045e+07,4.224672e+07,9.426240e+07,6.764843e+07,1.047854e+08,9.187730e+07,2.454824e+08,9.136357e+07,5.035357e+07,7.144303e+07,1.018912e+08
6,2004,1.526492e+08,6.554461e+07,5.892437e+07,8.679582e+07,3.861182e+08,8.713475e+07,7.158977e+07,4.074449e+07,8.886481e+07,6.722227e+07,1.202869e+08,9.497256e+07,2.576855e+08,9.212005e+07,5.867926e+07,6.889394e+07,1.057780e+08
7,2005,1.880066e+08,9.017177e+07,7.186869e+07,1.092691e+08,4.733628e+08,1.232707e+08,8.289254e+07,4.456601e+07,1.023897e+08,8.682448e+07,1.325403e+08,1.092375e+08,2.844098e+08,1.063099e+08,5.925987e+07,8.013367e+07,1.261478e+08
8,2006,1.919566e+08,8.747552e+07,6.908265e+07,9.865891e+07,4.550014e+08,1.222985e+08,7.064865e+07,4.181360e+07,1.012999e+08,9.191775e+07,1.276452e+08,1.106956e+08,2.807913e+08,1.125556e+08,4.847697e+07,7.578603e+07,1.227120e+08
9,2007,1.838475e+08,8.315305e+07,7.426453e+07,9.122505e+07,3.963170e+08,9.836996e+07,7.537253e+07,4.390015e+07,9.347274e+07,8.943993e+07,1.385520e+08,1.160225e+08,2.847280e+08,1.111433e+08,4.191020e+07,6.385455e+07,1.167984e+08


In [ ]:
actual_supply["west_half_lbs"] + actual_supply["east_half_lbs"] - (16 * demand["Average"])

0    -2.540108e+08
1    -1.125119e+08
2    -1.813819e+08
3    -3.393421e+07
4     2.042564e+07
5     1.174586e+08
6    -3.641517e+07
7    -3.828670e+08
8    -6.710320e+08
9    -3.743621e+08
10   -3.294263e+08
11   -3.584002e+08
12   -3.882762e+07
13   -2.132693e+08
14   -3.745184e+08
15   -2.297103e+08
16   -4.372103e+08
17             NaN
dtype: float64

In [57]:
orders_west.to_csv("/content/citrus_var_orders_west.csv", index=False)
orders_east.to_csv("/content/citrus_var_orders_east.csv", index=False)